# EDA Notebook — `Advanced_IoT_Dataset.csv`

Generated automated EDA notebook following the provided guidelines.

**Contents**:
1. Dataset Overview
2. Data Quality Checks
3. Data Cleaning
4. Descriptive Statistics
5. Transformation & Encoding
6. Outlier Detection & Treatment
7. Data Visualization
8. Insights & Next Steps

---

In [ ]:
# Imports and load dataset
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy import stats

plt.rcParams["figure.figsize"] = (8,5)

csv_path = r"/mnt/data/eda_workspace/Advanced_IoT_Dataset.csv"
df = pd.read_csv(csv_path)
print("Loaded:", csv_path)
print("Shape:", df.shape)
df.head()

## 1) Dataset Overview
- Show head/tail
- Shape, columns, dtypes
- Missing and unique counts

In [ ]:
# Overview: head, tail, shape, columns, dtypes, missing, unique counts
display(df.head())
display(df.tail())
print("Shape:", df.shape)
print("\nColumns and dtypes:")
print(df.dtypes)
print("\nMissing values (per column):")
print(df.isnull().sum())
print("\nPercentage missing:")
print((df.isnull().mean()*100).round(2))
print("\nUnique counts per column:")
print(df.nunique())

## 2) Data Quality Checks
- Duplicates
- Erroneous values (simple numeric checks)
- Formatting issues (string trimming)

In [ ]:
# 2) Data quality checks
# Duplicate rows
dupes = df.duplicated().sum()
print("Duplicate rows count:", dupes)

# Basic erroneous checks: For numeric columns, check for negative values if they should be non-negative.
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print("Numeric columns:", num_cols)

neg_counts = {}
for c in num_cols:
    neg = (df[c] < 0).sum()
    if neg>0:
        neg_counts[c]=int(neg)
print("Negative values per numeric column (if any):", neg_counts)

# Formatting issues: find leading/trailing spaces in object columns
obj_cols = df.select_dtypes(include=['object']).columns.tolist()
strip_issues = {}
for c in obj_cols:
    # count strings with leading/trailing spaces
    s = df[c].astype(str)
    mask = s.str.startswith(' ') | s.str.endswith(' ')
    cnt = mask.sum()
    if cnt>0:
        strip_issues[c]=int(cnt)
print("Columns with leading/trailing spaces (count):", strip_issues)

# Show first 5 rows of columns with issues (if any)
if strip_issues:
    cols = list(strip_issues.keys())
    display(df[cols].head())

## 3) Data Cleaning
- Handling missing values (demonstration)
- Removing duplicates
- Fix simple formatting issues (strip strings)

> NOTE: The notebook demonstrates typical steps — adapt imputation choices per domain knowledge.

In [ ]:
# 3) Data cleaning (demonstration)
df_clean = df.copy()

# Remove exact duplicate rows
before = df_clean.shape
df_clean = df_clean.drop_duplicates()
after = df_clean.shape
print(f"Dropped duplicates: before={before}, after={after}")

# Strip leading/trailing spaces in object columns
for c in df_clean.select_dtypes(include=['object']).columns:
    df_clean[c] = df_clean[c].astype(str).str.strip()

# For numeric columns: fill missing with median (demonstration)
for c in df_clean.select_dtypes(include=[np.number]).columns:
    if df_clean[c].isnull().sum() > 0:
        med = df_clean[c].median()
        df_clean[c] = df_clean[c].fillna(med)
        print(f"Filled NA in {c} with median={med}")

# For categorical/object columns: fill missing with mode
for c in df_clean.select_dtypes(include=['object']).columns:
    if df_clean[c].isnull().sum() > 0:
        mode = df_clean[c].mode().iloc[0] if not df_clean[c].mode().empty else ''
        df_clean[c] = df_clean[c].fillna(mode)
        print(f"Filled NA in {c} with mode='{mode}'")

print("\nAfter cleaning missing values and formatting:")
print(df_clean.isnull().sum())

## 4) Descriptive Statistics
- Mean, median, mode, min, max, variance, std, skewness, kurtosis
- Value counts for categorical columns

In [ ]:
# 4) Descriptive statistics
num = df_clean.select_dtypes(include=[np.number])
display(num.describe().T)

# Additional stats: variance, skew, kurtosis
stats_df = pd.DataFrame({
    'variance': num.var(),
    'skewness': num.skew(),
    'kurtosis': num.kurt()
})
display(stats_df)

# Mode for numeric (if needed) and value_counts for categorical
from collections import Counter
print("\nCategorical value counts (top 5) for object columns:")
for c in df_clean.select_dtypes(include=['object']).columns:
    print("\nColumn:", c)
    print(df_clean[c].value_counts().head(5))

## 5) Data Transformation & Encoding
- Scaling examples: MinMaxScaler and StandardScaler
- Example of One-Hot Encoding for a categorical column (if present)

In [ ]:
# 5) Transformation & Encoding
from sklearn.preprocessing import MinMaxScaler, StandardScaler

df_trans = df_clean.copy()

numeric_cols = df_trans.select_dtypes(include=[np.number]).columns.tolist()
print("Numeric columns:", numeric_cols)

# Apply MinMax scaler example on first up to 3 numeric columns (to keep output short)
cols_to_scale = numeric_cols[:3]
if cols_to_scale:
    scaler = MinMaxScaler()
    df_trans[cols_to_scale] = scaler.fit_transform(df_trans[cols_to_scale])
    print("Applied MinMaxScaler to:", cols_to_scale)
    display(df_trans[cols_to_scale].head())

# One-hot encode up to 2 categorical cols (if present)
cat_cols = df_trans.select_dtypes(include=['object']).columns.tolist()
if cat_cols:
    ohe_cols = cat_cols[:2]
    df_ohe = pd.get_dummies(df_trans, columns=ohe_cols, drop_first=True)
    print("One-hot encoded columns:", ohe_cols)
    print("New shape after OHE:", df_ohe.shape)
else:
    df_ohe = df_trans.copy()


## 6) Outlier Detection & Treatment
- IQR method and Z-score method demonstration
- Boxplot examples (matplotlib)

In [ ]:
# 6) Outlier detection
import numpy as np
from scipy import stats

df_out = df_clean.copy()
num_cols = df_out.select_dtypes(include=[np.number]).columns.tolist()

outlier_summary = {}
# IQR method
for c in num_cols:
    Q1 = df_out[c].quantile(0.25)
    Q3 = df_out[c].quantile(0.75)
    IQR = Q3 - Q1
    low = Q1 - 1.5 * IQR
    high = Q3 + 1.5 * IQR
    mask = (df_out[c] < low) | (df_out[c] > high)
    outlier_summary[c] = int(mask.sum())

print("Outliers detected by IQR (counts):")
print(outlier_summary)

# Z-score method: count points with |z| > 3
z_summary = {}
if num_cols:
    z_scores = np.abs(stats.zscore(df_out[num_cols], nan_policy='omit'))
    for i, c in enumerate(num_cols):
        z_summary[c] = int((z_scores[:, i] > 3).sum())
print("\nOutliers detected by Z-score (|z|>3) (counts):")
print(z_summary)

# Example boxplot for the first numeric column (if exists)
if num_cols:
    col = num_cols[0]
    plt.figure()
    plt.title(f"Boxplot for {col}")
    plt.boxplot(df_out[col].dropna())
    plt.ylabel(col)
    plt.show()

## 7) Data Visualization
- Univariate: histogram, boxplot
- Bivariate: scatter and correlation heatmap

(Using matplotlib only)

In [ ]:
# 7) Visualizations (matplotlib)
import matplotlib.pyplot as plt
num_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()

# Univariate histogram for up to 3 numeric cols
for c in num_cols[:3]:
    plt.figure()
    plt.title(f"Histogram of {c}")
    plt.hist(df_clean[c].dropna(), bins=30)
    plt.xlabel(c)
    plt.ylabel("Frequency")
    plt.show()

# Scatter plot for first two numeric columns (if available)
if len(num_cols) >= 2:
    x, y = num_cols[0], num_cols[1]
    plt.figure()
    plt.title(f"Scatter: {x} vs {y}")
    plt.scatter(df_clean[x], df_clean[y], s=8)
    plt.xlabel(x)
    plt.ylabel(y)
    plt.show()

# Correlation matrix heatmap (matplotlib)
if num_cols:
    corr = df_clean[num_cols].corr()
    plt.figure()
    plt.title("Correlation matrix")
    plt.imshow(corr, interpolation='nearest')
    plt.colorbar()
    plt.xticks(range(len(num_cols)), num_cols, rotation=90)
    plt.yticks(range(len(num_cols)), num_cols)
    plt.tight_layout()
    plt.show()

## 8) Insights & Next Steps
- Summarize brief findings and suggest next steps for modeling

**Save notebook**

In [ ]:
# 8) Quick automated insights (basic)
insights = []
# Missing values
mv = df.isnull().mean()
mv = mv[mv>0].sort_values(ascending=False)
if not mv.empty:
    insights.append("Columns with missing values: " + ", ".join(list(mv.index)))

# High cardinality categorical columns
for c in df.select_dtypes(include=['object']).columns:
    if df[c].nunique() > 50:
        insights.append(f"High cardinality: {c} ({df[c].nunique()} unique)")

# Numeric columns with many outliers (IQR > threshold)
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for c in num_cols:
    Q1 = df[c].quantile(0.25); Q3 = df[c].quantile(0.75); IQR = Q3-Q1
    if IQR == 0:
        continue
    out_pct = ((df[c] < (Q1 - 1.5*IQR)) | (df[c] > (Q3 + 1.5*IQR))).mean()
    if out_pct > 0.05:
        insights.append(f"Column {c} has {out_pct:.2%} outliers by IQR (>5%)")

print("Automated insights:")
for line in insights[:10]:
    print("-", line)

# Save a small cleaned sample as csv for reference
sample_path = Path("/mnt/data/eda_workspace/cleaned_sample.csv")
df_clean.sample(min(200, len(df_clean))).to_csv(sample_path, index=False)
print("\nSaved a cleaned sample to:", sample_path)

----
Notebook generation complete. The notebook file will be saved and is available for download.